<span style="color: #000508; font-family: Segoe UI; font-size: 1.2em; font-weight: 300;">🎯 Training Kernel: <strong><a href="https://www.kaggle.com/debarshichanda/pytorch-w-b-jigsaw-starter">[Pytorch + W&B] Jigsaw Starter</a></strong>.</span>


<div style = "padding : 20px; font-size : 16px; background-color: rgb(240, 245, 255); box-shadow: rgba(255, 255, 255, 0.2) 0px 0px 0px 1px inset, rgba(0, 0, 0, 0.9) 0px 0px 0px 1px;">

<span style = "font-size : 20px;">
Hi ! :)
    <br><br>
</span>
    
       
<div class="alert alert-warning">
    <b>CREDITS :</b> This notebook has been mainly written by <b>Debarshi Chanda</b>. Many thanks to him for his amazing work <a href="https://www.kaggle.com/c/jigsaw-toxic-severity-rating/discussion/286471"> (-> Link)</a> !
</div>
<br>
The initial score was <b><u>0.816</u></b> (LB). I added a little piece of code to obtain <b><u>0.817</u></b>. The optimization was worth being discussed, so I decided to make this notebook public. <br>
<a href="https://www.kaggle.com/c/jigsaw-toxic-severity-rating/discussion/287426">-> Link to the discussion of this topic</a>

The thing is : the private test dataset contains pairs of comments from the validation dataset. So if 3/3 commentators agreed on the classification on a pair of comments, we definitely should listen to them and adapt our rankings to fit their choices. That is, we must lower the score of a comment such that its new score becomes lower than every score of other comment that have been classified more toxic, and vice versa.

[Click here to see the implementation of this principle](#jump)

Tell me what you think of it !

<div style = "background-color: rgb(180,205,80); font-size : 20px;">
    And please <b>UPVOTE</b> if you liked the content, I would appreciate it a lot :)
</div>
    
</div>

In [ ]:
# adresses ip
#import re
#a = df.copy()
#"a['ip'] = a['text'].apply(lambda string : re.findall( r'[0-9]+(?:\.[0-9]+){3}', string))
#a[a['ip'].str.len()>=1]

In [ ]:
# smiley into text ? (brackets)
#import re
#a = df.copy()
#a['brackets'] = a['text'].apply(lambda string : re.findall(r"\[([A-Za-z0-9_]+)\]", string))
#a[a['brackets'].str.len()>=1]

In [ ]:
#df[df['text'].str.contains("FVCK")]

In [ ]:
#df[df['text'].str.contains("Your edits to")] # et les majuscules après

In [ ]:
#df_train = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")
#df_train[(df_train['less_toxic'].str.contains("Your edits to")) | (df_train['more_toxic'].str.contains("Your edits to"))] # et les majuscules après

In [ ]:
#df[(df['text'].str.contains("sockpuppet")) | (df['text'].str.contains("sock puppet"))]

In [ ]:
#df[df['text'].str.contains("STFU")]

In [ ]:
#df[(df['text'].str.contains("Re:")) | (df['text'].str.contains("Reply:")) | (df['text'].str.contains("RE:")) | (df['text'].str.contains("REDIRECT:"))]

In [ ]:
#df[df['text'].str.contains("User:")]

In [ ]:
#df[df['text'].str.contains("nonsense to Wikipedia")]

In [ ]:
#df[df['text'].str.contains("is considered vandalism")]

In [ ]:
#df[df['text'].str.contains("Please stop your disruptive editing")]

In [ ]:
import os
import gc
import cv2
import copy
import time
import random

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For Transformer Models
from transformers import AutoTokenizer, AutoModel

# Utils
from tqdm import tqdm

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
df.head()

In [ ]:
"""
# Création de paires arbitraires
nb = 1000

# Mélange
from sklearn.utils import shuffle
df_ = shuffle(df[['text']].drop_duplicates())
df_ = df_[~df_['text'].str.contains('#')]

# Création
paire1 = df_.sample(n = nb, random_state = 1).reset_index(drop=True)['text']
paire2 = df_.sample(n = nb, random_state = 2).reset_index(drop=True)['text']
paires = pd.DataFrame({'text1': paire1, 'text2': paire2})

# Résultat
print(paires.shape)
paires.head()

# Export CSV
paires.to_csv('paires_1000_test.csv', sep = '#', index=False)
"""

In [ ]:
CONFIG = dict(
    seed = 42,
    model_name = '../input/roberta-base',
    test_batch_size = 64,
    max_length = 128,
    num_classes = 1,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [ ]:
MODEL_PATHS = ['../input/pytorch-w-b-jigsaw-starter/Loss-Fold-0.bin',
               '../input/pytorch-w-b-jigsaw-starter/Loss-Fold-1.bin',
               '../input/pytorch-w-b-jigsaw-starter/Loss-Fold-2.bin',
               '../input/pytorch-w-b-jigsaw-starter/Loss-Fold-3.bin',
               '../input/pytorch-w-b-jigsaw-starter/Loss-Fold-4.bin']


"""
MODEL_PATHS = [ '../input/lossfold20211114/Loss-Fold-0.bin',
                '../input/lossfold20211114/Loss-Fold-1.bin',
                '../input/lossfold20211114/Loss-Fold-2.bin',
                '../input/lossfold20211114/Loss-Fold-3.bin',
                '../input/lossfold20211114/Loss-Fold-4.bin']

MODEL_PATHS = ['../input/k/vincentschuler/pytorch-w-b-jigsaw-starter/Loss-Fold-0.bin',
               '../input/k/vincentschuler/pytorch-w-b-jigsaw-starter/Loss-Fold-1.bin',
               '../input/k/vincentschuler/pytorch-w-b-jigsaw-starter/Loss-Fold-2.bin',
               '../input/k/vincentschuler/pytorch-w-b-jigsaw-starter/Loss-Fold-3.bin',
               '../input/k/vincentschuler/pytorch-w-b-jigsaw-starter/Loss-Fold-4.bin']
"""
print()

In [ ]:
import os
from datetime import datetime

for file_path in MODEL_PATHS :
    statbuf = os.stat(file_path)
    date_derniere_modif = statbuf.st_mtime
    date_derniere_modif = datetime.fromtimestamp(date_derniere_modif).strftime("%A, %B %d, %Y %I:%M:%S")
    print("Dernère modif de {}: {}".format(file_path, date_derniere_modif))

In [ ]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
# Traitement
def corrige_text(text) :
    # Suppression des sauts de lignes
    text = text.replace('\n', ' ')
    
    # Dico : 'vrai mot' : [mots à remplacer]
    dico_replace = {'fuck' : ['F U C K', 'F UCK', 'FU©K', 'F'],
                    'you' : ['U']}
    # Traitement
    for vrai_mot, liste in dico_replace.items() :
        for mot_a_remplacer in liste :
            for mot in (mot_a_remplacer.lower(), mot_a_remplacer.upper()) :
                if len(mot) > 1 : # attention aux lettres solos
                    text = text.replace(mot, vrai_mot)
                else :
                    text = ' '.join([x if x != mot else vrai_mot for x in text.split(' ')])
    
    # Résultat
    return text

# -------------------------

df['text'] = df['text'].apply(lambda text : corrige_text(text))

In [ ]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']        
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
test_dataset = JigsawDataset(df, CONFIG['tokenizer'], max_length=CONFIG['max_length'])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                         num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
class JigsawModel(nn.Module):
    def __init__(self, model_name):
        super(JigsawModel, self).__init__()
        self.model = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(768, CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        PREDS.append(outputs.view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = JigsawModel(CONFIG['model_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
preds = inference(MODEL_PATHS, test_loader, CONFIG['device'])

In [ ]:
print(f"Total Predictions: {preds.shape[0]}")
print(f"Total Unique Predictions: {np.unique(preds).shape[0]}")

In [ ]:
df['score'] = preds
df.head()

In [ ]:
df['score'] = df['score'].rank(method='first')
df.head()

In [ ]:
#df.drop('text', axis=1, inplace=True)
#df.to_csv("submission.csv", index=False)

____
# Correction of the scores of the comments whose pairs are present in the validation dataset <a id='jump'></a>

In [ ]:
# Open the file
df_train = pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

print('Dim AVANT :', df_train.shape)

In [ ]:
# If the pair has been ranked by multiple worker, we keep the order that is most unanimous
df_train['TEXT_ranked'] = df_train.apply(lambda row : row['less_toxic'] + ':' + row['more_toxic'], axis = 1)
df_train['TEXT_paire'] = df_train.apply(lambda row : min(row['less_toxic'], row['more_toxic']) + ':' + max(row['less_toxic'], row['more_toxic']), axis = 1)
df_train['Count_paire_ranked'] = df_train.groupby(['TEXT_ranked'])['TEXT_ranked'].transform('count')
df_train['Count_paire'] = df_train.groupby(['TEXT_paire'])['TEXT_ranked'].transform('count')
df_train['count_max'] = df_train.groupby(['TEXT_paire'])['Count_paire_ranked'].transform(max)

# Selection
df_train = df_train[df_train['Count_paire_ranked'] == df_train['count_max']]
df_train = df_train[df_train['Count_paire_ranked'] == 3] # every workers agreed

# Delete duplicates
df_train = df_train.drop(columns = ['worker'])
df_train = df_train.drop_duplicates()

# Results
df_train = df_train.sort_values(by = ['TEXT_ranked'])
df_train = df_train.drop(columns = ['Count_paire', 'Count_paire_ranked', 'count_max', 'TEXT_ranked', 'TEXT_paire']).drop_duplicates()
print('Dim APRES :', df_train.shape)
df_train.head()

In [ ]:
#!pip install openpyxl
#import openpyxl
#paires_toxic = pd.read_excel("../input/paires-excel/paires_toxic.xlsx")
#paires_toxic = paires_toxic[paires_toxic['MORE_TOXIC_1_ou_2'].isin([1, 2])]
#paires_toxic.to_csv('paires_toxic_CSV.csv', index=False, sep='#', encoding='utf-8')

#paires_toxic = pd.read_csv("../input/paires-csv-new/paires_toxic_CSV (2).csv", sep = '#', encoding = 'utf-8')

In [ ]:
# Ajout des données manuelles
paires_toxic = pd.read_csv("../input/paires-csv-new/paires_toxic_CSV (2).csv", sep = '#', encoding = 'utf-8')
paires_toxic = paires_toxic[paires_toxic['MORE_TOXIC_1_ou_2'].isin([1, 2])]

def less(row) :
    if row['MORE_TOXIC_1_ou_2'] == 1 : return row['text2']
    if row['MORE_TOXIC_1_ou_2'] == 2 : return row['text1']
def more(row) :
    if row['MORE_TOXIC_1_ou_2'] == 1 : return row['text1']
    if row['MORE_TOXIC_1_ou_2'] == 2 : return row['text2']
    
paires_toxic['less_toxic'] = paires_toxic.apply(lambda row : less(row), axis=1)
paires_toxic['more_toxic'] = paires_toxic.apply(lambda row : more(row), axis=1)
#paires_toxic['worker'] = pd.Series(['99999999']*len(paires_toxic))

paires_toxic = paires_toxic[['less_toxic', 'more_toxic']]
print("Nb paires manuelles :", paires_toxic.shape)

# Ajout des données manuelles
print("Nb paires manuelles AVANT :", df_train.shape)
df_train = df_train.append(paires_toxic)
print("Nb paires manuelles APRES :", df_train.shape)

In [ ]:
#df = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
#df['score'] = preds
#df['score'] = df['score'].rank(method='first')
#df.head()

In [ ]:
# Add score to the validation dataset
df_train = df_train.merge(df[['text', 'score']], left_on = 'less_toxic', right_on = 'text', how = 'left').drop_duplicates()
df_train = df_train.rename(columns = {'score' : 'score_less'})
df_train = df_train.drop(columns = ['text'])
df_train = df_train.merge(df[['text', 'score']], left_on = 'more_toxic', right_on = 'text', how = 'left').drop_duplicates()
df_train = df_train.rename(columns = {'score' : 'score_more'})
df_train = df_train.drop(columns = ['text'])

# Stats
df_train.head()

In [ ]:
# Stats
print(len(df_train[df_train['score_more'] < df_train['score_less']]), '/', len(df_train))

In [ ]:
# Test
df_train[df_train['score_more'] < df_train['score_less']].sort_values(['less_toxic'])

In [ ]:
# Correction of scores
df_train['score_max_du_less_toxic'] = df_train.groupby(['less_toxic'])['score_more'].transform(min) # score_min des textes + toxics
df_train['score_min_du_more_toxic'] = df_train.groupby(['more_toxic'])['score_less'].transform(max) # score_max des textes - toxics

# Join
df = df.merge(df_train[['less_toxic', 'score_less', 'score_max_du_less_toxic']], left_on = ['text', 'score'], right_on = ['less_toxic', 'score_less'], how = 'left')
df = df.drop(columns = ['less_toxic', 'score_less'])
df = df.merge(df_train[['more_toxic', 'score_more', 'score_min_du_more_toxic']], left_on = ['text', 'score'], right_on = ['more_toxic', 'score_more'], how = 'left')
df = df.drop(columns = ['more_toxic', 'score_more'])

# Rename
df = df.rename(columns = {'score_max_du_less_toxic' : 'borne_max', 'score_min_du_more_toxic' : 'borne_min'}) # le score doit est + petit que borne_max
df = df[['comment_id', 'text', 'score', 'borne_min', 'borne_max']].drop_duplicates()

# Aperçu
df.head()

In [ ]:
# CORRECTION DES SCORES
def corrige(row) :
    score, borne_min, borne_max = row['score'], row['borne_min'], row['borne_max']
    
    if not(pd.isna(borne_min)) and not(pd.isna(borne_max)) :
        if borne_max < borne_min : return (borne_max + borne_min ) // 2 # return score
        if score < borne_min : return borne_min+1
        if score > borne_max : return borne_max-1
        else :
            return score
        
    elif not(pd.isna(borne_min)) :
        if score < borne_min : return borne_min+1
        else : return score

    elif not(pd.isna(borne_max)) :
        if score > borne_max : return borne_max-1
        else : return score
        
    else :
        return score
    
# --------------------

# Application of correction
df['score_corrige'] = df.apply(lambda row : corrige(row), axis=1)
corrections = df[df['score'] != df['score_corrige']]
print("Nb of corrections : {}/{}.".format(len(corrections), len(df)))


# Show
corrections[['comment_id', 'text', 'score', 'score_corrige']]

In [ ]:
"""
# Création de paires du test set
nb = 1500

# Mélange
from sklearn.utils import shuffle
df_ = df[~df['text'].str.contains('#')]
df_ = df_[df_['text'].str.len() < 200]
df_ = df_.sort_values(by = ['score']).reset_index(drop=True)

# PAIRES GENTILLES
a = df_.iloc[:600]
paire1 = a.sample(n = nb//4, random_state = 1).reset_index(drop=True)
paire2 = a.sample(n = nb//4, random_state = 2).reset_index(drop=True)
gentilles = pd.DataFrame({})
gentilles['text1'] = paire1['text']
gentilles['text2'] = paire2['text']
gentilles['score1'] = paire1['score']
gentilles['score2'] = paire2['score']

# PAIRES NORMALES
a = df_.iloc[1000:1800]
paire1 = a.sample(n = nb//4, random_state = 1).reset_index(drop=True)
paire2 = a.sample(n = nb//4, random_state = 2).reset_index(drop=True)
normales = pd.DataFrame({})
normales['text1'] = paire1['text']
normales['text2'] = paire2['text']
normales['score1'] = paire1['score']
normales['score2'] = paire2['score']

# PAIRES MECHANTES
a = df_.iloc[2000:2800]
paire1 = a.sample(n = nb//4, random_state = 1).reset_index(drop=True)
paire2 = a.sample(n = nb//4, random_state = 2).reset_index(drop=True)
mechantes = pd.DataFrame({})
mechantes['text1'] = paire1['text']
mechantes['text2'] = paire2['text']
mechantes['score1'] = paire1['score']
mechantes['score2'] = paire2['score']

# PAIRES TOXIC
a = df_.iloc[3000:]
paire1 = a.sample(n = nb//4, random_state = 1).reset_index(drop=True)
paire2 = a.sample(n = nb//4, random_state = 2).reset_index(drop=True)
toxic = pd.DataFrame({})
toxic['text1'] = paire1['text']
toxic['text2'] = paire2['text']
toxic['score1'] = paire1['score']
toxic['score2'] = paire2['score']

# Création dataframe
df_paires_final = gentilles.append(normales).append(mechantes).append(toxic).reset_index(drop=True)

# Résultat
print(df_paires_final.shape)

# Export CSV
df_paires_final.to_csv('paires_test.csv', sep = '#', index=False)
df_paires_final.head()
"""
print()

In [ ]:
# Rank first
df = df[['comment_id', 'text', 'score_corrige']].drop_duplicates()
df['score'] = df['score_corrige'].rank(method='first')
df = df[['comment_id', 'text', 'score']].drop_duplicates()

# Show
df.head()

In [ ]:
df = df[['comment_id', 'score']].drop_duplicates()
print(df.shape)
df.to_csv("submission.csv", index=False)